# Anomaly Score comparison

In [1]:
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
sys.path.append(module_path)
import torch
from torchvision import datasets

from anomaly_scores.energy import energy_anomaly_score
from anomaly_scores.max_logit import max_logit_anomaly_score
from anomaly_scores.softmax import max_softmax_anomaly_score
from anomaly_scores.vim_scores import VIM
from energy_ood.CIFAR.models.wrn import WideResNet
from energy_ood.utils.svhn_loader import SVHN
from util import TEST_TRANSFORM
from vim_training.model import WideResVIMNet

from util.display_results import compare_all_results
from util.get_ood_score import get_ood_score_for_multiple_datasets
from vim_training.testing import test

## The data
Let's start with replicating the results from the paper. First, with the SVHN data set. 

In [2]:
loaders = []

id_data = datasets.CIFAR10("../data/cifar10", train=False, transform=TEST_TRANSFORM)
id_loader = torch.utils.data.DataLoader(
    id_data, batch_size=200, shuffle=False, num_workers=2, pin_memory=True
)
loaders.append(("CIFAR10", id_loader))


ood_data = SVHN(
    root="../data/svhn/",
    split="test",
    transform=TEST_TRANSFORM,
    download=False,
)
ood_loader = torch.utils.data.DataLoader(
    ood_data, batch_size=200, shuffle=True, num_workers=2, pin_memory=True
)
ood_num_examples = len(loaders[0][1].dataset) // 5
loaders.append(("SVHN", ood_loader))


data = datasets.CIFAR100("../data/cifar-100", train=False, transform=TEST_TRANSFORM)
loader = torch.utils.data.DataLoader(
    data, batch_size=200, shuffle=True, num_workers=2, pin_memory=True
)
loaders.append(("CIFAR100", loader))

## Models

We are using the Wide ResNet as in the paper.

In [3]:
models = []
model = WideResNet(depth=40, num_classes=10, widen_factor=2, dropRate=0.3)
model.load_state_dict(
    torch.load(
        "../energy_ood/CIFAR/snapshots/energy_ft/cifar10_wrn_s1_energy_ft_epoch_9.pt"
    )
)
model.eval()
_ = model.cuda()
models.append(("Energy ft", model))
model = WideResVIMNet(depth=40, num_classes=10, loader=loaders[0][1], widen_factor=2, dropRate=0.3, is_using_vim=True)
model.load_state_dict(
    torch.load(
        "../snapshots/vim_ft/CIFAR10_WRN_epoch_9.pt"
    )
)
model.eval()
_ = model.cuda()
models.append(("WRVM ft with VIM", model))

model = WideResVIMNet(depth=40, num_classes=10, loader=loaders[0][1], widen_factor=2, dropRate=0.3, is_using_vim=True)
model.load_state_dict(
    torch.load(
        "../snapshots/energy_vim_ft/CIFAR10_WRN_epoch_9.pt"
    )
)
model.eval()
_ = model.cuda()
models.append(("WRVM ft with Energy and with VIM", model))

# Anomaly Scores
Let's compare the scores.

In [4]:
import numpy as np

aurocs_results = {}
auprs_results = {}
all_results = []
for model_name, model in models:
    print(model_name)
    aurocs_results[model_name] = {}
    auprs_results[model_name] = {}

    if "with VIM" in model_name:
        vim_scoring = model.compute_anomaly_score
    else:
        vim = VIM(id_loader, model)
        vim_scoring = vim.compute_anomaly_score

    scores = [
        ("MaxLogit", max_logit_anomaly_score),
        ("MaxSoftmax", max_softmax_anomaly_score),
        ("Energy", energy_anomaly_score),
        ("VIM", vim_scoring),
    ]

    _, test_accuracy = test(model, loaders[0][1])
    aurocs_results[model_name]["test_acc"] = test_accuracy
    auprs_results[model_name]["test_acc"] = test_accuracy
    model_results = []

    for score_name, score in scores:
        print("  ", score_name)
        results = get_ood_score_for_multiple_datasets(
            loaders,
            model,
            score,
            is_using="last" if not score_name == "VIM" or "with VIM" in model_name else "last_penultimate",
            runs=3,
        )
        aurocs = [np.mean(aurocs) for aurocs, _, _ in results]
        aurocs.append(np.mean(aurocs))
        aurocs_results[model_name][score_name] = aurocs
        auprs = [np.mean(auprs) for _, auprs, _ in results]
        auprs.append(np.mean(auprs))
        auprs_results[model_name][score_name] = auprs
        model_results.append(results)
    all_results.append(model_results)

Energy ft
   MaxLogit
   MaxSoftmax
   Energy
   VIM
WRVM ft with VIM
   MaxLogit
   MaxSoftmax
   Energy
   VIM
WRVM ft with Energy and with VIM
   MaxLogit
   MaxSoftmax
   Energy
   VIM


In [5]:
compare_all_results(aurocs_results, loaders)

        Energy ft (4.85%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    99.13%    |    93.52%    |    96.33%   
               MaxSoftmax |    97.66%    |    92.11%    |    94.89%   
                   Energy |   *99.17%    |   *93.69%    |   *96.43%   
                      VIM |    64.86%    |    72.71%    |    68.79%   

 WRVM ft with VIM (5.05%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    53.70%    |    76.36%    |    65.03%   
               MaxSoftmax |    19.96%    |    52.01%    |    35.99%   
                   Energy |    91.45%    |    89.82%    |    90.64%   
                      VIM |   *98.98%    |   *93.66%    |   *96.32%   

WRVM ft with Energy and with VIM (5.42%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    98.55%    |   *94.01%    |    96.28%   
               MaxSoftmax |    17.49%    |    52.80%    |    35.15%   
                   Energy |    99.49%    |    93.93%    |   

In [6]:
compare_all_results(auprs_results, loaders)

        Energy ft (4.85%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    98.73%    |    93.63%    |    96.18%   
               MaxSoftmax |    96.94%    |    91.51%    |    94.22%   
                   Energy |   *98.77%    |   *93.79%    |   *96.28%   
                      VIM |    52.66%    |    67.16%    |    59.91%   

 WRVM ft with VIM (5.05%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    50.64%    |    73.43%    |    62.03%   
               MaxSoftmax |    35.97%    |    56.71%    |    46.34%   
                   Energy |    86.09%    |    87.28%    |    86.69%   
                      VIM |   *98.93%    |   *93.59%    |   *96.26%   

WRVM ft with Energy and with VIM (5.42%) |     SVHN     |   CIFAR100   |     AVG     
                 MaxLogit |    96.41%    |    93.68%    |    95.05%   
               MaxSoftmax |    34.10%    |    56.85%    |    45.48%   
                   Energy |    99.43%    |   *94.15%    |   

# Latex Prepartion

In [7]:
import numpy as np
from energy_ood.utils.display_results import get_measures

from util.get_ood_score import get_ood_scores

RUNS = 10
all_results = {}
for model_name, model in models:
    print(model_name)

    if "with VIM" in model_name:
        vim_scoring = model.compute_anomaly_score
    else:
        vim = VIM(id_loader, model)
        vim_scoring = vim.compute_anomaly_score

    scores = [
        ("MaxLogit", max_logit_anomaly_score),
        ("MaxSoftmax", max_softmax_anomaly_score),
        ("Energy", energy_anomaly_score),
        ("VIM", vim_scoring),
    ]

         

    # _, test_accuracy = test(model, loaders[0][1])

    ood_num_examples = len(loaders[0][1].dataset) // 5
    all_results[model_name] = {}
    for score_name, score in scores:    
        is_using="last" if not score_name == "VIM" or "with VIM" in model_name else "last_penultimate"
        print("  ", score_name)
        in_score = get_ood_scores(
            loaders[0][1],
            model,
            score,
            ood_num_examples,
            in_dist=True,
            is_using=is_using,
        )
        for ds_name, loader in loaders[1:]:
            if ds_name not in all_results[model_name].keys():
                all_results[model_name][ds_name] = {}
            print("    ", ds_name)
            all_results[model_name][ds_name][score_name] = {
                "AUROC": [],
                "AUPR": [],
                "FPR": [],
            }
            for _ in range(RUNS):
                out_score = get_ood_scores(
                    loader,
                    model,
                    score,
                    ood_num_examples,
                    is_using=is_using,
                )
                auroc, aupr, fpr = get_measures(out_score[:], in_score[:])
                all_results[model_name][ds_name][score_name]["AUROC"].append(auroc)
                all_results[model_name][ds_name][score_name]["AUPR"].append(aupr)
                all_results[model_name][ds_name][score_name]["FPR"].append(fpr)

Energy ft
   MaxLogit
     SVHN
     CIFAR100
   MaxSoftmax
     SVHN
     CIFAR100
   Energy
     SVHN
     CIFAR100
   VIM
     SVHN
     CIFAR100
WRVM ft with VIM
   MaxLogit
     SVHN
     CIFAR100
   MaxSoftmax
     SVHN
     CIFAR100
   Energy
     SVHN
     CIFAR100
   VIM
     SVHN
     CIFAR100
WRVM ft with Energy and with VIM
   MaxLogit
     SVHN
     CIFAR100
   MaxSoftmax
     SVHN
     CIFAR100
   Energy
     SVHN
     CIFAR100
   VIM
     SVHN
     CIFAR100


In [8]:
HAS_STD = False
MEASURES = ["AUROC"]

In [9]:
mean_std_results = []
for i in all_results.keys():
    res = []
    for j in all_results[i].keys():
        for k in all_results[i][j].keys():
            for l in MEASURES:
                res.append(np.mean(all_results[i][j][k][l]))
                if HAS_STD:
                    res.append(np.std(all_results[i][j][k][l]))
    mean_std_results.append(res)

In [10]:
import pandas as pd

col_stats = ["Mean"]
if HAS_STD:
    col_stats.append("Std")
cols = pd.MultiIndex.from_tuples(
    [
        (j, k, l, m)
        for i in [next(iter(all_results))]
        for j in all_results[i].keys()
        for k in all_results[i][j].keys()
        for l in MEASURES
        for m in col_stats
    ],
    names=["Dataset", "Score", "Measure", "Statistics"],
)
df = pd.DataFrame(np.array(mean_std_results)*100, columns=cols, index=all_results.keys()
)
stats = pd.DataFrame()
stats["Mean"] = df.mean(axis=0)
stats["Std"] = df.std(axis=0)
(df.style.highlight_max(axis=0, props='background-color:green;')
         .highlight_min(axis=0, props='background-color:red;'))